In [23]:
#Dependencies
import requests 
import pandas as pd
import numpy as np
import math
import os 
from statistics import mean, median, mode
%matplotlib inline
import matplotlib.pyplot as plt
# run Jupiter Notebook to call an API on the Realtor website 
%run Project_2_Mona.ipynb
# run Jupiter Notebook to scrape the Zolo website 
%run webScraping_zolo-mugu-Dec16.ipynb

Project_2_Mona.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
Project_2_Mona.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {


In [24]:
#Display the realtor Dataframe
maindfclean.head()

,Address,Sale Price,Mls Number,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated
0,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","283,900",W4652611,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,1,2019-12-15
1,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","294,500",W4653141,"Hospital, Public Transit",Condominium/Strata,Underground,1,1,2019-12-13
4,"#910 -2737 KEELE ST|Toronto, Ontario M3M2E9","339,700",W4641655,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,2,2019-12-05
5,"#907 -234 ALBION RD|Toronto, Ontario M9W6A5","355,000",W4622564,"Hospital, Park, Schools",Condominium/Strata,Underground,1,2,2019-12-09
7,"#601 -3555 BATHURST ST|Toronto, Ontario M6A2Y8","365,000",C4637991,"Park, Public Transit, Schools",Condominium/Strata,Underground,2,2,2019-12-06


In [25]:
#Count the number of records identified
maindfclean.count()

Address               778
Sale Price            778
Mls Number            778
Near By Ammenities    778
Ownership Type        778
Parking               778
Bathrooms             778
Bedrooms              778
Date Updated          778
dtype: int64

In [26]:
#maindfclean.set_index('Mls Number')
#maindfclean.head()

In [27]:
# parse the address string on the left of "|" which gives us the Street Number and Name
maindfclean['Street']=maindfclean['Address'].str.split('|').str[0]
#parse the address string on the right of "|" which gives us the City and Province and Postal code
maindfclean['City - Province']=maindfclean['Address'].str.split('|').str[1]
#drop the postal code from teh above string
maindfclean['City Province']=maindfclean['City - Province'].str[:-6]
# parse the City Province string on the left of ", which gives us the City Name
maindfclean['City']=maindfclean['City Province'].str.split(',').str[0]
# parse the City Province string on the right  of ", which gives us the Province 
maindfclean['Province']=maindfclean['City Province'].str.split(',').str[1]
# parse the last 6 digits of the full address to capture the Postal code
maindfclean['Postal_Code']=maindfclean['Address'].str[-6:]
#print the DataFrame
maindfclean.head()

C:\Users\pasqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\pasqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\pasqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

,Address,Sale Price,Mls Number,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated,Street,City - Province,City Province,City,Province,Postal_Code
0,"#628 -2737 KEELE ST|Toronto, Ontario M3M2E9","283,900",W4652611,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,1,2019-12-15,#628 -2737 KEELE ST,"Toronto, Ontario M3M2E9","Toronto, Ontario",Toronto,Ontario,M3M2E9
1,"#614 -2737 KEELE ST|Toronto, Ontario M3M2E9","294,500",W4653141,"Hospital, Public Transit",Condominium/Strata,Underground,1,1,2019-12-13,#614 -2737 KEELE ST,"Toronto, Ontario M3M2E9","Toronto, Ontario",Toronto,Ontario,M3M2E9
4,"#910 -2737 KEELE ST|Toronto, Ontario M3M2E9","339,700",W4641655,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,2,2019-12-05,#910 -2737 KEELE ST,"Toronto, Ontario M3M2E9","Toronto, Ontario",Toronto,Ontario,M3M2E9
5,"#907 -234 ALBION RD|Toronto, Ontario M9W6A5","355,000",W4622564,"Hospital, Park, Schools",Condominium/Strata,Underground,1,2,2019-12-09,#907 -234 ALBION RD,"Toronto, Ontario M9W6A5","Toronto, Ontario",Toronto,Ontario,M9W6A5
7,"#601 -3555 BATHURST ST|Toronto, Ontario M6A2Y8","365,000",C4637991,"Park, Public Transit, Schools",Condominium/Strata,Underground,2,2,2019-12-06,#601 -3555 BATHURST ST,"Toronto, Ontario M6A2Y8","Toronto, Ontario",Toronto,Ontario,M6A2Y8


In [28]:
#drop the Column 0 -"Full Address", Column 10 = City-Province and Column 11 - City Province
df = maindfclean.drop(maindfclean.columns[[0, 10,11]], axis=1)   
# rint the new DataFRame
df.head()

,Sale Price,Mls Number,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated,Street,City,Province,Postal_Code
0,"283,900",W4652611,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,1,2019-12-15,#628 -2737 KEELE ST,Toronto,Ontario,M3M2E9
1,"294,500",W4653141,"Hospital, Public Transit",Condominium/Strata,Underground,1,1,2019-12-13,#614 -2737 KEELE ST,Toronto,Ontario,M3M2E9
4,"339,700",W4641655,"Hospital, Park, Public Transit, Schools",Condominium/Strata,Underground,1,2,2019-12-05,#910 -2737 KEELE ST,Toronto,Ontario,M3M2E9
5,"355,000",W4622564,"Hospital, Park, Schools",Condominium/Strata,Underground,1,2,2019-12-09,#907 -234 ALBION RD,Toronto,Ontario,M9W6A5
7,"365,000",C4637991,"Park, Public Transit, Schools",Condominium/Strata,Underground,2,2,2019-12-06,#601 -3555 BATHURST ST,Toronto,Ontario,M6A2Y8


In [29]:
#Display the Zolo Dataframe
house_df.head()

,Age,Days on Site,Listed By,Lot Size,MLS®#,Size,Style,Taxes,Type,Walk Score,address,city,hoot,price,Business Type,Property Type,Zoning,Strata Fees,Year Built
0,0-5 years,"5 (Dec 11, 2019)","Zolo Realty, Brokerage",70.00 x 115 Feet,C4651523,5000 sqft,3-Storey,"$10,920 /yr",Detached,20,28 Caswell Drive,Toronto,Willowdale East,"$3,888,888",NaN,NaN,NaN,NaN,NaN
1,No Data,"20 (Nov 26, 2019)","Zolo Realty, Brokerage",33.00 x 120 Feet,E4642347,No Data,2-Storey,"$3,115 /yr",Detached,77,18 Vanbrugh Avenue,Toronto,Birchcliffe-cliffside,"$799,000",NaN,NaN,NaN,NaN,NaN
2,0-5 years,"27 (Nov 19, 2019)","Zolo Realty, Brokerage",32.00 x 107 Feet,E4638092,1500-2000 sqft,2-Storey,"$4,038 /yr",Detached,62,53 Northbrook Road,Toronto,East York,"$1,699,000",NaN,NaN,NaN,NaN,NaN
3,51-99 years,"1 (Dec 16, 2019)","Royal Star Realty Inc., Brokerage",50.00 x 110 Feet,W4654296,2000-2500 sqft,Backsplit 4,"$3,246 /yr",Detached,53,3 Beaconhill Road,Toronto,Mount Olive-silverstone-jamestown,"$995,400",NaN,NaN,NaN,NaN,NaN
4,No Data,"1 (Dec 16, 2019)","Jdl Realty Inc., Brokerage",56.50 x 145 Feet,C4654304,No Data,2-Storey,"$10,077 /yr",Detached,49,30 Hedgewood Road,Toronto,Bridle Path-sunnybrook-york Mills,"$2,889,000",NaN,NaN,NaN,NaN,NaN


In [30]:
# new data frame with split value columns 
new = house_df["Days on Site"].str.split("(", n = 1, expand = True) 
# making separate Days column from new data frame 
house_df["Days"]= new[0] 
# making separate Date posted on MLScolumn from new data frame 
house_df["Date posted on MLS"]= new[1].str[:-1] 
#house_df["Date posted on MLS"].str[:-1]
# Dropping old Name columns 
house_df.drop(columns =["Days on Site","Days"], inplace = True) 
#rename the MLS number column
house_df.rename(columns={'MLS®#':'Mls Number'}, inplace=True)
# df display 
house_df

,Age,Listed By,Lot Size,Mls Number,Size,Style,Taxes,Type,Walk Score,address,city,hoot,price,Business Type,Property Type,Zoning,Strata Fees,Year Built,Date posted on MLS
0,0-5 years,"Zolo Realty, Brokerage",70.00 x 115 Feet,C4651523,5000 sqft,3-Storey,"$10,920 /yr",Detached,20,28 Caswell Drive,Toronto,Willowdale East,"$3,888,888",NaN,NaN,NaN,NaN,NaN,"Dec 11, 2019"
1,No Data,"Zolo Realty, Brokerage",33.00 x 120 Feet,E4642347,No Data,2-Storey,"$3,115 /yr",Detached,77,18 Vanbrugh Avenue,Toronto,Birchcliffe-cliffside,"$799,000",NaN,NaN,NaN,NaN,NaN,"Nov 26, 2019"
2,0-5 years,"Zolo Realty, Brokerage",32.00 x 107 Feet,E4638092,1500-2000 sqft,2-Storey,"$4,038 /yr",Detached,62,53 Northbrook Road,Toronto,East York,"$1,699,000",NaN,NaN,NaN,NaN,NaN,"Nov 19, 2019"
3,51-99 years,"Royal Star Realty Inc., Brokerage",50.00 x 110 Feet,W4654296,2000-2500 sqft,Backsplit 4,"$3,246 /yr",Detached,53,3 Beaconhill Road,Toronto,Mount Olive-silverstone-jamestown,"$995,400",NaN,NaN,NaN,NaN,NaN,"Dec 16, 2019"
4,No Data,"Jdl Realty Inc., Brokerage",56.50 x 145 Feet,C4654304,No Data,2-Storey,"$10,077 /yr",Detached,49,30 Hedgewood Road,Toronto,Bridle Path-sunnybrook-york Mills,"$2,889,000",NaN,NaN,NaN,NaN,NaN,"Dec 16, 2019"
5,No Data,"Right At Home Realty Inc., Brokerage",57.50 x 143.93 Feet,W4654216,No Data,Bungalow,"$3,936 /yr",Detached,68,1023 Kipling Avenue,Toronto,Islington-city Centre West,"$1,550,000",NaN,NaN,NaN,NaN,NaN,"Dec 16, 2019"
6,No Data,"One Percent Realty Ltd., Brokerage",55.96 x 189.2 Feet,E4654229,No Data,2-Storey,"$5,215 /yr",Detached,45,82 Whistling Hills Drive,Toronto,Milliken,"$1,198,000",NaN,NaN,NaN,NaN,NaN,"Dec 16, 2019"
7,No Data,"Re/max All-stars Realty Inc., Brokerage",100.00 x 160 Feet,E4654204,No Data,2-Storey,"$7,651 /yr",Detached,25,18 Hill Crescent,Toronto,Scarborough Village,"$1,995,000",NaN,NaN,NaN,NaN,NaN,"Dec 16, 2019"
8,No Data,"Re/max Realtron Realty Inc., Brokerage",40.00 x 189 Feet,E4654174,No Data,Bungalow,"$3,085 /yr",Detached,59,118 Citadel Drive,Toronto,Bendale,"$788,000",NaN,NaN,NaN,NaN,NaN,"Dec 16, 2019"
9,No Data,"Kamali Group Realty, Brokerage",45.51 x 158.32 Feet,E4654166,No Data,2-Storey,"$4,112 /yr",Detached,69,92 Midland Avenue,Toronto,Birchcliffe-cliffside,"$800,000",NaN,NaN,NaN,NaN,NaN,"Dec 16, 2019"


In [31]:
merge_table = pd.merge(house_df, df, on="Mls Number")
merge_table

,Age,Listed By,Lot Size,Mls Number,Size,Style,Taxes,Type,Walk Score,address,...,Near By Ammenities,Ownership Type,Parking,Bathrooms,Bedrooms,Date Updated,Street,City,Province,Postal_Code
0,No Data,"Sutton Group-admiral Realty Inc., Brokerage",25.00 x 125 Feet,C4653945,2000-2500 sqft,2-Storey,No Data,Detached,93,273 Manor Road,...,"Park, Schools",Freehold,Garage,5,4 + 1,2019-12-16,273 MANOR RD E,Toronto,Ontario,M4S1S3
1,No Data,"Sutton Group-admiral Realty Inc., Brokerage",25.00 x 125 Feet,C4653945,2000-2500 sqft,2-Storey,No Data,Detached,93,273 Manor Road,...,"Park, Schools",Freehold,Garage,5,4 + 1,2019-12-16,273 MANOR RD E,Toronto,Ontario,M4S1S3
2,No Data,"Ipro Realty Ltd., Brokerage",44.00 x 30 Feet,W4653555,No Data,2-Storey,"$2,548 /yr",Detached,57,67 Innes Avenue,...,"Park, Public Transit, Schools",Freehold,Detached garage,2,4 + 3,2019-12-15,67 INNES AVE,Toronto,Ontario,M6E1N2
3,No Data,"Ipro Realty Ltd., Brokerage",44.00 x 30 Feet,W4653555,No Data,2-Storey,"$2,548 /yr",Detached,57,67 Innes Avenue,...,"Park, Public Transit, Schools",Freehold,Detached garage,2,4 + 3,2019-12-15,67 INNES AVE,Toronto,Ontario,M6E1N2
4,No Data,"Tony K Real Estate, Brokerage",25.00 x 109.37 Feet,E4653432,No Data,Bungalow,"$2,948 /yr",Detached,75,127 Leyton Avenue,...,"Park, Public Transit, Schools",Freehold,Attached garage,2,3 + 3,2019-12-14,127 LEYTON AVE,Toronto,Ontario,M1L3V3
5,51-99 years,"Century 21 Atria Realty Inc., Brokerage",27.00 x 94.5 Feet,W4653183,1100-1500 sqft,Bungalow,"$2,400 /yr",Detached,36,22 Blakley Avenue,...,"Public Transit, Schools",Freehold,Detached garage,2,2 + 1,2019-12-13,22 BLAKLEY AVE,Toronto,Ontario,M6N3Y5
6,No Data,"Royal Lepage Real Estate Services Ltd., Brokerage",60.00 x 125 Feet,W4653158,2500-3000 sqft,Bungalow-Raised,"$6,663 /yr",Detached,67,57 Wimbleton Road,...,"Park, Public Transit, Schools",Freehold,Carport,2,3 + 1,2019-12-14,57 WIMBLETON RD,Toronto,Ontario,M9A3S3
7,51-99 years,"Real Estate Homeward, Brokerage",40.00 x 215 Feet,E4652989,No Data,Bungalow,"$3,736 /yr",Detached,31,41 Florens Avenue,...,"Park, Public Transit",Freehold,Garage,2,2 + 1,2019-12-15,41 FLORENS AVE,Toronto,Ontario,M1L1R7
8,0-5 years,"Sam Mcdadi Real Estate Inc., Brokerage",30.00 x 100 Feet,W4652563,1500-2000 sqft,2-Storey,No Data,Detached,64,45 William Street,...,"Park, Public Transit, Schools",Freehold,Detached garage,4,3 + 1,2019-12-16,45 WILLIAM ST,Toronto,Ontario,M9N2G6
9,51-99 years,"Homelife Best-seller Realty Inc., Brokerage",28.50 x 105 Feet,E4652561,No Data,Bungalow,"$2,593 /yr",Detached,60,26 Jeavons Avenue,...,"Park, Public Transit, Schools",Freehold,Detached garage,2,2 + 3,2019-12-13,26 JEAVONS AVE,Toronto,Ontario,M1K1S8
